# Introduction and Basics of Chatbots and Conversational Agents

In this notebook, we work on the following topics:
- A direct API call from OpenAI to perform chat completions
- Chat completions with Langchain APIs
- Using prompt template to create custom prompt
- Using output parser to extract key information from the completion

We are going to covers samples from the following use case:
Travel and Hospitality

In [4]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("..")

In [6]:
# import openai
import pprint

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

import sys
module_dir = os.path.abspath('src')  # Gets the absolute path to the src directory
sys.path.append(module_dir)
from helper_functions import llm_completion

In [14]:
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

# Loading Parameters

In [18]:
modelID = "gpt-3.5-turbo"

# API call from OpenAI

In [32]:
question = "Why do cats always land on their feet?"
completion = llm_completion(question, model=modelID)
nprint(completion)

('Cats have a unique ability called the "righting reflex" that allows them to '
 'twist their bodies in mid-air to land on their feet when they fall. This '
 'reflex is a combination of their flexible spine, inner ear balance, and keen '
 'sense of orientation. It helps them to quickly adjust their position and '
 'land safely on their feet, minimizing the impact of the fall.')


## Working on the completion Style and System

In [33]:
sys_content = "You are a technical expert in Animal Physiology and explain everything in scientific terms."
completion = llm_completion(question, model=modelID, sys_content=sys_content)
nprint(completion)

('Cats have a unique ability to consistently land on their feet when falling '
 'from a height, a phenomenon known as the "righting reflex." This behavior is '
 'a result of their highly developed vestibular system, which includes the '
 'inner ear structures responsible for balance and orientation in space.\n'
 '\n'
 'When a cat falls, it quickly orients its body to face downward by using its '
 'vestibular system to detect changes in position and acceleration. This '
 "triggers a series of rapid and coordinated movements in the cat's body, "
 'including twisting of the spine and limbs, to reposition itself in mid-air '
 'and land on its feet.\n'
 '\n'
 'Additionally, cats have a flexible skeletal structure and a specialized '
 'collarbone that allows them to twist their bodies mid-air without losing '
 'balance. This agility, combined with their keen sense of spatial awareness '
 'and reflexes, enables cats to consistently land on their feet and minimize '
 'the impact of a fall.\n'


Now lets change the style of the answer

In [37]:
style = "The answer must be understabdable for a 5 year old child."
prompt = f"""{question} 
{style}
"""
print(prompt)

completion = llm_completion(prompt, model=modelID, sys_content=sys_content)
nprint(completion)

Why do cats always land on their feet? 
The answer must be understabdable for a 5 year old child.

('Cats have a special ability called the "righting reflex" that helps them '
 'land on their feet when they fall. This reflex is a combination of their '
 'flexible spine, strong muscles, and keen sense of balance. When a cat falls, '
 'it quickly twists its body in mid-air to orient itself upright before '
 'landing on its feet. This helps them avoid injury and land safely on the '
 'ground. So, cats are like little acrobats who can always land on their feet!')


Let's make it even more simpler for the child

In [38]:
sys_content = "You are a cool 5 years old child with funny sense of humor."

completion = llm_completion(prompt, model=modelID, sys_content=sys_content)
nprint(completion)

('Well, you see, cats have a special superpower called "cat magic" that helps '
 "them twist their bodies in the air and land on their feet. It's like they "
 'have a secret cat trampoline hidden inside them! So, no matter how high they '
 'fall from, they always end up on their paws. Cats are just really good at '
 'acrobatics!')


Let's make it even more creative by increasing the tempreature of the LLM model to 2.0.

In [39]:
completion = llm_completion(prompt, model=modelID, sys_content=sys_content, temperature=2.0)
nprint(completion)

('Because cats are magical creatures with invisible parachutes on their backs '
 'that float them down gently whenever they fall! How cool is that? 😺✨🪂')


# Using Langchain APIs to perform chat completions
Now we perform the above chat completions using Langchain modules which are more simpler and easier to use.  
We use prompt template to create the prompt with different style.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [19]:
QAchat = ChatOpenAI(temperature=0, model=modelID)
template = """{sys_content}
Answer this following question:  
{question} 
Style: {style}
"""
prompt_template = ChatPromptTemplate.from_template(template)
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


input_variables=['question', 'style', 'sys_content'] template='{sys_content}\nAnswer this following question:  \n{question} \nStyle: {style}\n'
['question', 'style', 'sys_content']


In [71]:
prompt = prompt_template.format_messages(
                    sys_content = sys_content,
                    style=style,
                    question=question)
print(prompt)
print("The formated prompt is:\n")
nprint(prompt[0].content)

[HumanMessage(content='You are a cool 5 years old child with funny sense of humor.\nAnswer this following question:  \nWhy do cats always land on their feet? \nStyle: The answer must be understabdable for a 5 year old child.\n')]
The formated prompt is:

('You are a cool 5 years old child with funny sense of humor.\n'
 'Answer this following question:  \n'
 'Why do cats always land on their feet? \n'
 'Style: The answer must be understabdable for a 5 year old child.\n')


In [73]:
completion = QAchat(prompt)
nprint(completion.content)

('Because they have special cat magnets in their paws that help them always '
 'land on their feet!')


# Using output parser
Here we use output parser to extract key information from the completion.  
We define a custom format instructions to let LLM know what information must be extracted from the completion.
We need this information to be extracted as a JSON object, so we can use it as a dictionary.

**Use case**: Travel and Hospitality

In [23]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

Here is a sample query from the user from which we extract the key information about the trip.

In [12]:
customer_query = """\
We're planning a family vacation to Hawaii in December. \
We need recommendations for family-friendly hotels and activities suitable for kids. \
Also, any travel advisories we should be aware of?\
Our budget for the entire trip is around $5000.
"""

We use ResponseSchema to define how the key information must be extracted from the completion.

In [34]:
destination_schema = ResponseSchema(name="destination",
                             description="What is the destination of the trip?")
travel_date_schema = ResponseSchema(name="travel_date",
                                      description="When is the trip planned for?")
requests_schema = ResponseSchema(name="requests",
                                    description="""Extract any specific requests made by the traveler,
                                    and output them as a comma separated Python list.""")
budget_schema = ResponseSchema(name="budget",
                             description="""What is the budget for the entire trip? 
                                If this information is not found, output -1.""")

completion_schemas = [destination_schema, 
                    travel_date_schema,
                    requests_schema,
                    budget_schema]
output_parser = StructuredOutputParser.from_response_schemas(completion_schemas)        
# output_parser
format_instructions = output_parser.get_format_instructions()            
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // What is the destination of the trip?
	"travel_date": string  // When is the trip planned for?
	"requests": string  // Extract any specific requests made by the traveler,
                                    and output them as a comma separated Python list.
	"budget": string  // What is the budget for the entire trip? 
                                If this information is not found, output -1.
}
```


In [10]:
vacation_template = """\
For the following query, extract the following information:

destination: What is the destination of the trip?\

travel_date: When is the trip planned for?\

requests: Extract any specific requests made by the traveler,\
and output them as a comma separated Python list.

budget: What is the budget for the entire trip? \
If this information is not found, output -1.

Format the output as JSON with the following keys:
destination
travel_date
requests
budget

query: {query}

{format_instructions}
"""
prompt_template = ChatPromptTemplate.from_template(vacation_template)
print(prompt_template)

input_variables=['query'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='For the following query, extract the following information:\n\ndestination: What is the destination of the trip?\ntravel_date: When is the trip planned for?\nrequests: Extract any specific requests made by the traveler,and output them as a comma separated Python list.\n\nbudget: What is the budget for the entire trip? If this information is not found, output -1.\n\nFormat the output as JSON with the following keys:\ndestination\ntravel_date\nrequests\nbudget\n\nquery: {query}\n'))]


In [15]:
prompt = prompt_template.format_messages(query = customer_query)
print(prompt)
print("The formated prompt is:\n")
nprint(prompt[0].content)

[HumanMessage(content="For the following query, extract the following information:\n\ndestination: What is the destination of the trip?\ntravel_date: When is the trip planned for?\nrequests: Extract any specific requests made by the traveler,and output them as a comma separated Python list.\n\nbudget: What is the budget for the entire trip? If this information is not found, output -1.\n\nFormat the output as JSON with the following keys:\ndestination\ntravel_date\nrequests\nbudget\n\nquery: We're planning a family vacation to Hawaii in December. We need recommendations for family-friendly hotels and activities suitable for kids. Also, any travel advisories we should be aware of?Our budget for the entire trip is around $5000.\n\n")]
The formated prompt is:

('For the following query, extract the following information:\n'
 '\n'
 'destination: What is the destination of the trip?\n'
 'travel_date: When is the trip planned for?\n'
 'requests: Extract any specific requests made by the trave

In [21]:
completion = QAchat(prompt)
print(completion.content)

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{
    "destination": "Hawaii",
    "travel_date": "December",
    "requests": ["recommendations for family-friendly hotels", "recommendations for family-friendly activities suitable for kids", "travel advisories"],
    "budget": 5000
}
